In [107]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm  = ChatOpenAI(model_name="gpt-4o-mini",
                temperature=0.1,
                streaming=True,
                )

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input" : input },{"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Nicolas, I live in South Korea", "Wow That is so cool!")
add_message("South Korea is so pretty", "I wannd go there")

get_history()

{'history': 'The human introduces himself as Nicolas and mentions that he lives in South Korea. The AI responds positively, expressing excitement about this information. The human comments that South Korea is very pretty, and the AI expresses a desire to visit.'}